Importing data

In [8]:
library(coin)
library(lme4)
library(lmerTest)
library(plyr)
library(xtable)
library(LMERConvenienceFunctions)
library(car)
# library(lmerTest) see also https://link.springer.com/article/10.3758/s13428-016-0809-y 

source("r_utils/mer-utils.R")
source("r_utils/regression-utils.R")

In [18]:
data_path <- file.path(readLines(file("data_path.txt", open="r")), "processed", "choices.txt")
output_table_path <- file.path(readLines(file("table_path.txt", open="r")))

choice.data = read.table(data_path, sep="\t", header=T)

Warning message in readLines(file("data_path.txt", open = "r")):
"incomplete final line found on 'data_path.txt'"Warning message in readLines(file("table_path.txt", open = "r")):
"incomplete final line found on 'table_path.txt'"

A bit of preprocessing to create variables for analyzing sequential effects

In [3]:
preprocess_data <- function(choice.data) {
    # any exclusions?
    stats.df = choice.data
    stats.df$subj_id = as.factor(stats.df$subj_id)

    # generate trial number
    max.trial = max(stats.df$trial_no)
    max.block = max(stats.df$block_n)
    stats.df$all_trial_no = (stats.df$session_no-1) * (max.block * max.trial) +
      (stats.df$block_no-1) * max.trial +
      stats.df$trial_no

    contrasts(stats.df$is_com) = contr.sum(n=2) /2 *-1
    contrasts(stats.df$is_com)# true is pos
    
#     stats.df = stats.df[(stats.df$RT < 1.5),]

    str(stats.df)

    return(stats.df)
}

stats.df = preprocess_data(choice.data)

'data.frame':	26280 obs. of  29 variables:
 $ subj_id        : Factor w/ 11 levels "196","247","269",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ session_no     : int  1 1 1 1 1 1 1 1 1 1 ...
 $ block_no       : int  1 1 1 1 1 1 1 1 1 1 ...
 $ trial_no       : int  1 2 3 4 5 6 7 8 9 10 ...
 $ is_practice    : Factor w/ 2 levels "False","True": 1 1 1 1 1 1 1 1 1 1 ...
 $ direction      : num  180 180 0 180 0 180 0 180 180 180 ...
 $ coherence      : num  0.256 0.064 0.064 0.064 0 0.032 0.256 0.512 0.512 0.064 ...
 $ duration       : int  800 800 800 800 800 800 800 800 800 800 ...
 $ response       : int  180 0 180 180 180 0 180 0 180 180 ...
 $ trial_time     : num  0.83 1.33 1.21 1.3 1.2 ...
 $ is_correct     : Factor w/ 2 levels "False","True": 2 1 1 2 1 1 1 1 2 2 ...
 $ xflips         : int  0 1 2 0 0 1 1 0 0 0 ...
 $ max_d          : num  -99.9 99.5 -51.8 58.9 48.7 ...
 $ idx_max_d      : num  51 75 71 59 64 52 53 44 20 108 ...
 $ midline_d      : num  0 0 0 0 0 0 1 0 1 0 ...
 $ idx_midline_d  :

# Analysis 1. Accuracy as a function of coherence

In [4]:
rnd_effects_analysis_1 <- function(stats.df){
    rnd1.lmer = glmer(is_correct ~ (1|subj_id), 
                 stats.df[stats.df$coherence!=0,], 
                 family = binomial)

    # rnd intercept for each participant and random slope of coherence
    # diff avg acc, diff coherence effect for each p
    rnd2.lmer = glmer(is_correct ~ (c.(coherence)|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff linear and quad learning effect for each p
    rnd3.lmer = glmer(is_correct ~ (poly(coherence, 2, raw = T)|subj_id), 
                           stats.df[stats.df$coherence!=0,], 
                           family = binomial)

    # rnd.lmer with com and coherence
    rnd4.lmer = glmer(is_correct ~ ((is_com + c.(coherence))|subj_id),
                      stats.df[stats.df$coherence!=0,],
                      family = binomial)

    # rnd.lmer with com by coherence
    rnd5.lmer = glmer(is_correct ~ ((is_com*c.(coherence))|subj_id),
                      stats.df[stats.df$coherence!=0,],
                      family = binomial)
    
    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [5]:
rnd_effects_analysis_1(stats.df)

boundary (singular) fit: see ?isSingular
boundary (singular) fit: see ?isSingular


Data: stats.df[stats.df$coherence != 0, ]
Models:
rnd1.lmer: is_correct ~ (1 | subj_id)
rnd2.lmer: is_correct ~ (c.(coherence) | subj_id)
rnd3.lmer: is_correct ~ (poly(coherence, 2, raw = T) | subj_id)
rnd4.lmer: is_correct ~ ((is_com + c.(coherence)) | subj_id)
rnd5.lmer: is_correct ~ ((is_com * c.(coherence)) | subj_id)
          Df   AIC   BIC   logLik deviance    Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  2 23814 23830 -11904.9    23810                               
rnd2.lmer  4 20088 20120 -10039.9    20080 3729.989      2     <2e-16 ***
rnd3.lmer  7 20097 20153 -10041.4    20083    0.000      3          1    
rnd4.lmer  7 20086 20142 -10036.2    20072   10.519      0     <2e-16 ***
rnd5.lmer 11 19982 20070  -9979.8    19960  112.852      4     <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd5.lmer"
[1] "Best model according to BIC"
[1] "rnd5.lmer"


rnd5 is the best, but the resulting full model does not converge, just as with rnd4, therefore we only use random effect of coherence (rnd2)

In [14]:
run_analysis_1 <- function(stats.df){
    choice.mer = glmer(is_correct ~ is_com*c.(coherence) + ((c.(coherence))|subj_id),
                      stats.df[stats.df$coherence!=0,],
                      family = binomial)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients
    row.names(choice.output) <- c("Intercept", "Is CoM", "Coherence", "Is CoM by Coherence")

    file_name = paste(output_table_path, "is_correct_vs_coh.tex", sep="")
    print(xtable(choice.output, digits = c(4,4,4,4,4),# display = c("g","g","g","g","g"), 
                 label = "tab:is_correct_vs_coh",
                 caption = "Parameters of a linear mixed-effects model analysing choice accuracy 
                            as a function of coherence and presence or absence of a change-of-mind. The model 
                            included random intercept and random slope for coherence."), 
          caption.placement = "top", table.placement="t", floating.environment = "table*",
          math.style.exponents = TRUE, type = "latex", booktabs = TRUE, file = file_name)
}

In [15]:
run_analysis_1(stats.df)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_correct ~ is_com * c.(coherence) + ((c.(coherence)) | subj_id)
   Data: stats.df[stats.df$coherence != 0, ]

     AIC      BIC   logLik deviance df.resid 
 19959.8  20015.7  -9972.9  19945.8    21888 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-68.293   0.015   0.257   0.680   1.107 

Random effects:
 Groups  Name          Variance Std.Dev. Corr
 subj_id (Intercept)   0.3047   0.552        
         c.(coherence) 8.2189   2.867    0.98
Number of obs: 21895, groups:  subj_id, 11

Fixed effects:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)             1.1751     0.1645   7.144 9.06e-13 ***
is_com1                -1.3436     0.1129 -11.905  < 2e-16 ***
c.(coherence)           6.2918     0.8918   7.055 1.72e-12 ***
is_com1:c.(coherence)  -6.7564     0.8058  -8.385  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001

# Analysis 2. Probability of CoM as a function of initiation time

In [13]:
rnd_effects_analysis_2 <- function(stats.df){
    rnd1.lmer = glmer(is_com ~ (1|subj_id), stats.df, family = binomial)

    rnd2.lmer = glmer(is_com ~ (c.(coherence)|subj_id), stats.df, family = binomial)

    rnd3.lmer = glmer(is_com ~ (poly(c.(coherence), 2, raw = T)|subj_id), stats.df, family = binomial)

    rnd4.lmer = glmer(is_com ~ (c.(RT)|subj_id), stats.df, family = binomial)
    
    rnd5.lmer = glmer(is_com ~ (c.(RT) + c.(coherence)|subj_id), stats.df, family = binomial)
    
    rnd6.lmer = glmer(is_com ~ (c.(RT)*c.(coherence)|subj_id), stats.df, family = binomial)
    
    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer, rnd6.lmer)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [14]:
rnd_effects_analysis_2(stats.df)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00240658 (tol = 0.001, component 1)"

Data: stats.df
Models:
rnd1.lmer: is_com ~ (1 | subj_id)
rnd2.lmer: is_com ~ (c.(coherence) | subj_id)
rnd4.lmer: is_com ~ (c.(RT) | subj_id)
rnd3.lmer: is_com ~ (poly(c.(coherence), 2, raw = T) | subj_id)
rnd5.lmer: is_com ~ (c.(RT) + c.(coherence) | subj_id)
rnd6.lmer: is_com ~ (c.(RT) * c.(coherence) | subj_id)
          Df    AIC    BIC  logLik deviance   Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  2 6682.8 6699.1 -3339.4   6678.8                              
rnd2.lmer  4 6518.7 6551.4 -3255.4   6510.7 168.055      2  < 2.2e-16 ***
rnd4.lmer  4 6479.5 6512.2 -3235.8   6471.5  39.202      0  < 2.2e-16 ***
rnd3.lmer  7 6502.5 6559.7 -3244.2   6488.5   0.000      3          1    
rnd5.lmer  7 6390.7 6447.9 -3188.3   6376.7 111.816      0  < 2.2e-16 ***
rnd6.lmer 11 6370.7 6460.6 -3174.4   6348.7  27.967      4  1.267e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd6.lmer"
[1] "Best model according to BIC"
[1] "rnd5.lmer"

Model with rnd6 does not converge, so we chose rnd5 for random effects structure

In [16]:
run_analysis_2 <- function(stats.df, exp_name){
    choice.mer = glmer(is_com ~ c.(RT) + c.(coherence) + (c.(RT) + c.(coherence)|subj_id), stats.df, family = binomial)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients
    row.names(choice.output) <- c("Intercept", "RT", "Coherence")

    file_name = paste(output_table_path, "is_com_vs_RT.tex", sep="")
    print(xtable(choice.output, digits = c(4,4,4,4,4),
                 label = "tab:is_com_vs_RT",
                 caption = "Parameters of a linear mixed-effects model analysing probability of a change-of-mind 
                            as a function of coherence and response time. The model included random intercept and random 
                            slopes for response time and coherence."), 
          caption.placement = "top", table.placement="t", floating.environment = "table*",
          math.style.exponents = TRUE, type = "latex", booktabs = TRUE, file = file_name)
}

In [19]:
run_analysis_2(stats.df)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_com ~ c.(RT) + c.(coherence) + (c.(RT) + c.(coherence) | subj_id)
   Data: stats.df

     AIC      BIC   logLik deviance df.resid 
  6369.1   6442.7  -3175.6   6351.1    26271 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1639 -0.2028 -0.1337 -0.0859 28.7279 

Random effects:
 Groups  Name          Variance Std.Dev. Corr       
 subj_id (Intercept)   1.025    1.013               
         c.(RT)        2.090    1.446    -0.04      
         c.(coherence) 1.597    1.264     0.50  0.65
Number of obs: 26280, groups:  subj_id, 11

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.9960     0.3147 -12.699  < 2e-16 ***
c.(RT)          1.6534     0.4760   3.473 0.000514 ***
c.(coherence)  -3.1224     0.6097  -5.121 3.03e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlatio

# Analysis 3. Response time as a function of coherence

In [26]:
rnd_effects_analysis_3 <- function(stats.df){
    rnd1.lmer = lmer(RT ~ (1|subj_id), stats.df)

    rnd2.lmer = lmer(RT ~ (c.(coherence)|subj_id), stats.df)

    rnd3.lmer = lmer(RT ~ (is_correct|subj_id), stats.df)

    rnd4.lmer = lmer(RT ~ ((c.(coherence)+is_correct)|subj_id), stats.df)
    
    rnd5.lmer = lmer(RT ~ ((c.(coherence)*is_correct)|subj_id), stats.df)
    
    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer)
#     rnd.anova = anova(rnd4.lmer, rnd5.lmer)
    print(rnd.anova)
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [27]:
rnd_effects_analysis_3(stats.df)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.168253 (tol = 0.002, component 1)"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.0312311 (tol = 0.002, component 1)"refitting model(s) with ML (instead of REML)


Data: stats.df
Models:
rnd1.lmer: RT ~ (1 | subj_id)
rnd2.lmer: RT ~ (c.(coherence) | subj_id)
rnd3.lmer: RT ~ (is_correct | subj_id)
rnd4.lmer: RT ~ ((c.(coherence) + is_correct) | subj_id)
rnd5.lmer: RT ~ ((c.(coherence) * is_correct) | subj_id)
          Df   AIC   BIC  logLik deviance   Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  3 16180 16205 -8087.1    16174                              
rnd2.lmer  5 11160 11201 -5574.8    11150 5024.58      2     <2e-16 ***
rnd3.lmer  5 14548 14589 -7269.1    14538    0.00      0          1    
rnd4.lmer  8 10863 10928 -5423.3    10847 3691.59      3     <2e-16 ***
rnd5.lmer 12 10667 10765 -5321.7    10643  203.27      4     <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd5.lmer"
[1] "Best model according to BIC"
[1] "rnd5.lmer"


rnd2 is the best random effects structure at which the model with fixed effects converges

In [42]:
run_analysis_3 <- function(stats.df, exp_name){
    choice.mer = lmer(RT ~ (c.(coherence)*is_correct + (c.(coherence)|subj_id)), stats.df)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients

    row.names(choice.output) <- c("Intercept", "Coherence", "Is correct", "Coherence by Is correct")

    file_name = paste(output_table_path, "RT_vs_coh.tex", sep="")
    print(xtable(choice.output, digits = c(4,4,4,4,4,4),
                 label = "tab:RT_vs_coh",
                 caption = "Parameters of a linear mixed-effects model analysing response time as
                            a function of coherence and choice correctness. The model included random intercept and random 
                            slope for coherence."), 
          caption.placement = "top", table.placement="t", floating.environment = "table*",
          math.style.exponents = TRUE, type = "latex", booktabs = TRUE, file = file_name)
}

In [43]:
run_analysis_3(stats.df)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: RT ~ (c.(coherence) * is_correct + (c.(coherence) | subj_id))
   Data: stats.df

REML criterion at convergence: 10762.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.3492 -0.4627 -0.0536  0.3817  9.8433 

Random effects:
 Groups   Name          Variance Std.Dev. Corr 
 subj_id  (Intercept)   0.04608  0.2147        
          c.(coherence) 0.33269  0.5768   -0.83
 Residual               0.08770  0.2961        
Number of obs: 26280, groups:  subj_id, 11

Fixed effects:
                               Estimate Std. Error         df t value Pr(>|t|)
(Intercept)                   2.353e-01  6.498e-02  1.014e+01   3.622  0.00457
c.(coherence)                 7.315e-02  1.794e-01  1.125e+01   0.408  0.69114
is_correctTrue               -1.232e-01  6.145e-03  2.626e+04 -20.040  < 2e-16
c.(coherence):is_correctTrue -6.067e-01  4.565e-02  2.626e+04 -13.289  < 2e-16
                    

# Difference between initial and final decisions in CoM trials

In [44]:
run_binom_test_all <- function(data){
    binom.test(x = nrow(data[(data$is_correct=='True'),]), n = nrow(data), 
           p = 0.5, alternative = "greater", conf.level = 0.95)
}

In [45]:
run_binom_test_all(stats.df[stats.df$is_com=='True',])


	Exact binomial test

data:  nrow(data[(data$is_correct == "True"), ]) and nrow(data)
number of successes = 428, number of trials = 775, p-value = 0.002014
alternative hypothesis: true probability of success is greater than 0.5
95 percent confidence interval:
 0.5221211 1.0000000
sample estimates:
probability of success 
             0.5522581 


We can also test this separately for each coherence level

In [46]:
run_binom_test <- function(data, coherence){
    binom.test(x = nrow(data[(data$is_correct=='True') & (data$coherence==coherence),]), 
           n = nrow(data[data$coherence==coherence,]), 
           p = 0.5, alternative = "greater", conf.level = 0.95)
}

In [47]:
run_binom_test(stats.df[stats.df$is_com=='True',], coherence=0.032)


	Exact binomial test

data:  nrow(data[(data$is_correct == "True") & (data$coherence == coherence),  and nrow(data[data$coherence == coherence, ])    ]) and nrow(data[data$coherence == coherence, ])
number of successes = 95, number of trials = 175, p-value = 0.1449
alternative hypothesis: true probability of success is greater than 0.5
95 percent confidence interval:
 0.4778698 1.0000000
sample estimates:
probability of success 
             0.5428571 


In [48]:
run_binom_test(stats.df[stats.df$is_com=='True',], coherence=0.064)


	Exact binomial test

data:  nrow(data[(data$is_correct == "True") & (data$coherence == coherence),  and nrow(data[data$coherence == coherence, ])    ]) and nrow(data[data$coherence == coherence, ])
number of successes = 80, number of trials = 172, p-value = 0.8392
alternative hypothesis: true probability of success is greater than 0.5
95 percent confidence interval:
 0.4004792 1.0000000
sample estimates:
probability of success 
             0.4651163 


In [49]:
run_binom_test(stats.df[stats.df$is_com=='True',], coherence=0.128)


	Exact binomial test

data:  nrow(data[(data$is_correct == "True") & (data$coherence == coherence),  and nrow(data[data$coherence == coherence, ])    ]) and nrow(data[data$coherence == coherence, ])
number of successes = 94, number of trials = 155, p-value = 0.004966
alternative hypothesis: true probability of success is greater than 0.5
95 percent confidence interval:
 0.5375526 1.0000000
sample estimates:
probability of success 
             0.6064516 


In [50]:
run_binom_test(stats.df[stats.df$is_com=='True',], coherence=0.256)


	Exact binomial test

data:  nrow(data[(data$is_correct == "True") & (data$coherence == coherence),  and nrow(data[data$coherence == coherence, ])    ]) and nrow(data[data$coherence == coherence, ])
number of successes = 56, number of trials = 79, p-value = 0.0001318
alternative hypothesis: true probability of success is greater than 0.5
95 percent confidence interval:
 0.6134122 1.0000000
sample estimates:
probability of success 
             0.7088608 


In [51]:
run_binom_test(stats.df[stats.df$is_com=='True',], coherence=0.512)


	Exact binomial test

data:  nrow(data[(data$is_correct == "True") & (data$coherence == coherence),  and nrow(data[data$coherence == coherence, ])    ]) and nrow(data[data$coherence == coherence, ])
number of successes = 14, number of trials = 24, p-value = 0.2706
alternative hypothesis: true probability of success is greater than 0.5
95 percent confidence interval:
 0.3967851 1.0000000
sample estimates:
probability of success 
             0.5833333 
